Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [10]:
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

http://vta2.amigocloud.com/api/otp/plan?mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.4002813%2C-121.9396906&toPlace=37.326763%2C-121.856533&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=15&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE

In [68]:
def query_string(start_lat, start_long, dest_lat, dest_long, bus = True, train = True, walk_dist = 5000, walk_reluc = 2, visual = False):
    base = 'http://vta2.amigocloud.com'
    api = '/api/otp/plan?'
    date = 'date=2016-07-12'
    time = ''
    time = '&time=07%3A0'
    
    if bus == True or train == True:
        mode = '&mode=WALK%2C'
        if bus == True:
            mode = mode + 'BUSISH%2C'
        if train == True:
            mode = mode + 'TRAINISH'
    #mode = '&mode=WALK%2CBUSISH%2CTRAINISH'
    
    start = '&fromPlace=' + str(start_lat) + '%2C' + str(start_long)
    #start = '&fromPlace=37.389268%2C-121.879746'
    dest = '&toPlace=' + str(dest_lat) + '%2C' + str(dest_long)
    #dest = '&toPlace=37.4031595%2C-121.9865994'
    num_iter = '&numItineraries=3'
    
    walk = '&maxWalkDistance=' + str(walk_dist) + '&'
    
    bike = 'bikeSpeed=10'
    
    walk_reluctance = '&walkReluctance=' + str(walk_reluc)
    max_tran = '&maxTransfers=5'
    triangle = '&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE'
    arriveby = '&arriveBy=false'
    
    #Visualize doesn't work right at the moment
    if visual == True:
        api = '/planner?'
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluctance + max_tran + triangle + arriveby
    else:
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluctance + max_tran + triangle + arriveby

    return query

In [69]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')
known_starts=pd.read_csv('Known_Starts.csv')
known_dests=pd.read_csv('Known_Dests.csv')

In [70]:
df = pd.DataFrame();

#df['start_lat'] = rand_starts['Lat']
#df['start_long'] = rand_starts['Long']
#df['dest_lat'] = rand_dests['Lat']
#df['dest_long'] = rand_dests['Long']

df['start_lat'] = known_starts['Lat']
df['start_long'] = known_starts['Long']
df['start_loc']= known_starts['Desc']
df['dest_lat'] = known_dests['Lat']
df['dest_long'] = known_dests['Long']
df['dest_loc'] = known_dests['Desc']

In [71]:
df.head()
#Random From and To pints

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose


In [72]:
#visuals = []
#visuals.append(query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row], walk_dist = 10000,visual = True))
#df['visual'] = visuals

In [73]:
def input_params(df, max_walk, walk_reluc_factor):
    row = 0
    times=[]
    results = []
    queries = []

    trips = []
    lowest = []
    
    for each in df.iterrows():
        query = query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], 
                             dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row],
                             walk_dist = max_walk, walk_reluc = walk_reluc_factor)
        queries.append(query)
        r = requests.get(query)
        query_json = json.loads(r.text)
        results.append(str(query_json))
        one_set_of_times= query_json['debugOutput']['pathTimes']
        times.append(one_set_of_times)
        row = row + 1
    df['times'] = times
    
    
    #Calculate Lowest trip length
    for each in df['times']:
        trips.append(len(each))
        if len(each) != 0:
            lowest.append(min(each))
        else:
            lowest.append(999)
    df['trips']=trips
    df['lowest']=lowest
    
    df['queries'] = queries
    df['results'] = results
    
    
    return df

In [74]:
series=input_params(df, max_walk = 5000, walk_reluc_factor = 2)

In [57]:
series

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc,times,trips,lowest,queries,results
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA,"[114, 121, 124]",3,114,http://vta2.amigocloud.com/api/otp/plan?date=2...,"{'plan': {'from': {'lon': -121.868908, 'lat': ..."
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose,"[49, 40, 49]",3,40,http://vta2.amigocloud.com/api/otp/plan?date=2...,"{'plan': {'from': {'lon': -121.93975519745304,..."


In [67]:
series

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc,times,trips,lowest,queries,results
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA,"[115, 127, 143]",3,115,http://vta2.amigocloud.com/api/otp/plan?date=2...,"{'plan': {'from': {'lon': -121.868908, 'lat': ..."
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose,"[33, 50, 51]",3,33,http://vta2.amigocloud.com/api/otp/plan?date=2...,"{'plan': {'from': {'lon': -121.93975519745304,..."


In [75]:
series

,start_lat,start_long,start_loc,dest_lat,dest_long,dest_loc,times,trips,lowest,queries,results
0,37.394535,-121.868908,"Capitol Avenue @ Camino del Rey, San Jose Park...",37.400281,-121.939691,3331 N. 1st St VTA,"[114, 126, 143]",3,114,http://vta2.amigocloud.com/api/otp/plan?date=2...,"{'plan': {'from': {'lon': -121.868908, 'lat': ..."
1,37.400281,-121.939691,3331 N. 1st St VTA,37.331997,-121.887286,Fairmont San Jose,"[46, 57, 45]",3,45,http://vta2.amigocloud.com/api/otp/plan?date=2...,"{'plan': {'from': {'lon': -121.93975519745304,..."


In [30]:
df.to_csv('series.csv')

In [ ]:
#'http://docs.python-guide.org/en/latest/scenarios/json/'
#query_json['plan']['itineraries'][x]['legs'][y]['legGeometry']

In [ ]:
#final = pd.DataFrame([each, every, sum(df['lowest']), sum(df['trips'])], columns = ['walk_dist','walk_reluc','lowest','trips'])
#finals.append(final)
#finals

In [1]:
import polyline

'_p~iF~ps|U_ulL~ugC_hgN~eq`@'

In [58]:
df['queries'][0]

'http://vta2.amigocloud.com/api/otp/plan?date=2016-07-12&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.394535%2C-121.868908&toPlace=37.400281%2C-121.939691&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=2&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false'

In [90]:
query= df['queries'][0]

In [77]:
from datetime import datetime

In [79]:
datetime.fromtimestamp(1468332000000/1000)

datetime.datetime(2016, 7, 12, 7, 0)

In [80]:
datetime.fromtimestamp(1468333015000/1000)

datetime.datetime(2016, 7, 12, 7, 16, 55)

In [81]:
datetime.fromtimestamp(1468332028000/1000)

datetime.datetime(2016, 7, 12, 7, 0, 28)

In [82]:
datetime.fromtimestamp(1468333420000/1000)

datetime.datetime(2016, 7, 12, 7, 23, 40)

In [84]:
datetime.fromtimestamp(1468333420000/1000).time()

datetime.time(7, 23, 40)

In [88]:
1392.0/60.0

23.2

In [93]:
datetime.fromtimestamp(1468333420000/1000)-datetime.fromtimestamp(1468332028000/1000)

datetime.timedelta(0, 1392)

In [94]:
print(_)

0:23:12


In [89]:
query

NameError: name 'query' is not defined

In [91]:
query

'http://vta2.amigocloud.com/api/otp/plan?date=2016-07-12&time=07%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.394535%2C-121.868908&toPlace=37.400281%2C-121.939691&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=2&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false'

In [ ]:
r = requests.get(query)
query_json = json.loads(r.text)
results.append(str(query_json))
one_set_of_times= query_json['debugOutput']['pathTimes']
times.append(one_set_of_times)
